# 感知器（单层感知器） 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class Perceptron:
    def __init__(self, n_inputs=2):
        self.W = np.random.randn(n_inputs,1)
        self.b = np.random.randn(1)
    def forward(self, x):
        # 实际上相当于使用了阶跃函数作为激活函数
        return ((np.matmul(x.reshape((1,-1)),self.W.reshape((-1,1)))+self.b) > 0).astype(np.int32)
    
    def backward(self, x, label, pre, alpha=0.05):
        self.W = self.W + alpha * (label - pre) * x.reshape(*self.W.shape)
        self.b -= 0

In [6]:
data_x = np.array([[0,0],[0,1],[1,0],[1,1]])

## 与门 

In [3]:
class Andgate(Perceptron):
    def __init__(self):
        self.W = np.array([[0.5],[0.5]])
        self.b = -0.6

In [8]:
and_gate = Andgate()
for x in data_x:
    print x,and_gate.forward(x).squeeze()

[0 0] 0
[0 1] 0
[1 0] 0
[1 1] 1


## 或门 

In [4]:
class Orgate(Perceptron):
    def __init__(self):
        self.W = np.array([[0.5],[0.5]])
        self.b = -0.4

In [9]:
or_gate = Orgate()
for x in data_x:
    print x,or_gate.forward(x).squeeze()

[0 0] 0
[0 1] 1
[1 0] 1
[1 1] 1


## 与非门 

In [5]:
class Nandgate(Perceptron):
    def __init__(self):
        self.W = np.array([[-0.5],[-0.5]])
        self.b = 0.6

In [10]:
nand_gate = Nandgate()
for x in data_x:
    print x,nand_gate.forward(x).squeeze()

[0 0] 1
[0 1] 1
[1 0] 1
[1 1] 0


## 异或门 

In [13]:
class Xorgate(Perceptron):
    def __init__(self):
        pass
    def forward(self, x):
        and_gate = Andgate()
        or_gate = Orgate()
        nand_gate = Nandgate()
        
        or_res = or_gate.forward(x)
        nand_res = nand_gate.forward(x)
        return and_gate.forward(np.array([or_res, nand_res]))

In [14]:
xor_gate = Xorgate()
for x in data_x:
    print x,xor_gate.forward(x).squeeze()

[0 0] 0
[0 1] 1
[1 0] 1
[1 1] 0


## 感知器学习 

感知器学习的策略是最小化如下的损失函数
$$L_{(W,b)} = -\sum_{X_{i} \in M}y_{i}(WX_{i}+b)$$
其中M表示错误分类的实例集合，$WX_{i}+b$表示$X_{i}$到超平面$WX+b=0$的距离，且该距离与$y_{i}$符号相反，所以需要添加一个`-`号；为了最小化这个损失函数为0，即所有实例均被正确分类，我们采用SGD来进行最小化。
